In [1]:
from os.path import abspath, dirname, exists, join
repository_folder = dirname(dirname(abspath('.')))
configuration_path = join(repository_folder, 'development.ini')
assert exists(configuration_path)

In [2]:
from pyramid.paster import bootstrap
env = bootstrap(configuration_path)
env

{'root': <pyramid.traversal.DefaultRootFactory at 0x7fbef5af1e90>,
 'closer': <function pyramid.scripting.prepare.<locals>.closer()>,
 'registry': <Registry asset_tracker>,
 'request': <Request at 0x7fbef5af1d50 GET http://localhost/>,
 'root_factory': pyramid.traversal.DefaultRootFactory,
 'app': <pyramid.router.Router at 0x7fbef5aee310>}

In [3]:
# Get database connection
# request = env['request']
# db = request.db
# db

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

database_engine = create_engine('sqlite://', echo=True)
get_database_session = sessionmaker()
get_database_session.configure(bind=database_engine)
db = get_database_session()

In [5]:
from asset_tracker.models.meta import Base
Base.metadata.create_all(database_engine)

2020-02-05 18:52:54,419 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-05 18:52:54,420 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:52:54,422 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-05 18:52:54,424 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:52:54,425 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("asset")
2020-02-05 18:52:54,426 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:52:54,428 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("bus")
2020-02-05 18:52:54,433 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:52:54,437 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("connection")
2020-02-05 18:52:54,439 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:52:54,441 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("line_type")
2020-02-05 18:52:54,442 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:52:54,446 INFO sqlalchemy

In [6]:
# Load JSON
import json
datasets_folder = join(repository_folder, 'datasets')
assets_json_path = join(datasets_folder, 'assets1.json')
assets_geojson_path = join(datasets_folder, 'assets1.geojson')

In [7]:
asset_dictionaries = json.load(open(assets_json_path, 'rt'))
asset_geometries = json.load(open(assets_geojson_path, 'rt'))

In [8]:
asset_dictionaries

[{'id': '1', 'typeCode': 's', 'name': 'Substation A'},
 {'id': '2',
  'typeCode': 't',
  'name': 'Substation Transformer 1',
  'attributes': {'vendorName': 'Schneider Electric',
   'productName': 'HVT36A',
   'productVersion': '2.0.1',
   'phaseCount': 3,
   'windingCount': 2,
   'winding1Winding2PercentReactance': 0.008},
  'connections': [{'busId': 'Ai',
    'attributes': {'connectionType': 'delta',
     'baseVoltage': 115,
     'baseVoltageUnit': 'kV',
     'power': 5000,
     'powerUnit': 'kVA',
     'powerPercentResistance': 0.0005}},
   {'busId': 'Ao',
    'attributes': {'connectionType': 'wye',
     'baseVoltage': 4.16,
     'baseVoltageUnit': 'kV',
     'power': 5000,
     'powerUnit': 'kVA',
     'powerPercentResistance': 0.0005}}]},
 {'id': '3',
  'typeCode': 'm',
  'name': 'Industrial Meter 1',
  'attributes': {'vendorName': 'Schneider Electric',
   'productName': 'SCH-MV10',
   'productVersion': '10.5.7',
   'phaseCount': 3,
   'loadModel': 1},
  'connections': [{'busId': '

In [9]:
from asset_tracker.models import *

In [10]:
# Print asset_id for each asset
for asset_d in asset_dictionaries:
    print(asset_d['id'])

1
2
3
4


In [11]:
db

In [12]:
import transaction

with transaction.manager:
    print(db.query(Asset).count())

2020-02-05 18:52:54,571 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 18:52:54,575 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset) AS anon_1
2020-02-05 18:52:54,578 INFO sqlalchemy.engine.base.Engine ()
0


In [13]:
# Check if asset id exists in database for each asset
for asset_d in asset_dictionaries:
    asset_id = asset_d['id']
    asset = db.query(Asset).first()
    print(asset)

2020-02-05 18:52:54,592 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset
 LIMIT ? OFFSET ?
2020-02-05 18:52:54,594 INFO sqlalchemy.engine.base.Engine (1, 0)
None
2020-02-05 18:52:54,597 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset
 LIMIT ? OFFSET ?
2020-02-05 18:52:54,602 INFO sqlalchemy.engine.base.Engine (1, 0)
None
2020-02-05 18:52:54,606 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, 

In [14]:
db.query(Asset).count()

2020-02-05 18:52:54,624 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset) AS anon_1
2020-02-05 18:52:54,627 INFO sqlalchemy.engine.base.Engine ()


0

In [15]:
asset_dictionaries[0]

{'id': '1', 'typeCode': 's', 'name': 'Substation A'}

In [16]:
asset_dictionaries[1]

{'id': '2',
 'typeCode': 't',
 'name': 'Substation Transformer 1',
 'attributes': {'vendorName': 'Schneider Electric',
  'productName': 'HVT36A',
  'productVersion': '2.0.1',
  'phaseCount': 3,
  'windingCount': 2,
  'winding1Winding2PercentReactance': 0.008},
 'connections': [{'busId': 'Ai',
   'attributes': {'connectionType': 'delta',
    'baseVoltage': 115,
    'baseVoltageUnit': 'kV',
    'power': 5000,
    'powerUnit': 'kVA',
    'powerPercentResistance': 0.0005}},
  {'busId': 'Ao',
   'attributes': {'connectionType': 'wye',
    'baseVoltage': 4.16,
    'baseVoltageUnit': 'kV',
    'power': 5000,
    'powerUnit': 'kVA',
    'powerPercentResistance': 0.0005}}]}

In [17]:
print(len(asset_dictionaries))

4


In [18]:
asset_d = asset_dictionaries[1]

In [19]:
asset_d['attributes']

{'vendorName': 'Schneider Electric',
 'productName': 'HVT36A',
 'productVersion': '2.0.1',
 'phaseCount': 3,
 'windingCount': 2,
 'winding1Winding2PercentReactance': 0.008}

In [20]:
asset_d['connections']

[{'busId': 'Ai',
  'attributes': {'connectionType': 'delta',
   'baseVoltage': 115,
   'baseVoltageUnit': 'kV',
   'power': 5000,
   'powerUnit': 'kVA',
   'powerPercentResistance': 0.0005}},
 {'busId': 'Ao',
  'attributes': {'connectionType': 'wye',
   'baseVoltage': 4.16,
   'baseVoltageUnit': 'kV',
   'power': 5000,
   'powerUnit': 'kVA',
   'powerPercentResistance': 0.0005}}]

In [21]:
from asset_tracker.models import AssetTypeCode

In [22]:
if {}: print('hey')

In [23]:
import pickle
pickle.dumps(None)

b'\x80\x03N.'

In [24]:
import pickle
pickle.dumps({})

b'\x80\x03}q\x00.'

In [25]:
# Check if asset id exists in database for each asset
asset_id_by_temporary_id = {}

for asset_d in asset_dictionaries:
    asset_id = asset_d['id']
    try:
        asset_type_code = AssetTypeCode(asset_d['typeCode'])
    except ValueError:
        # !!! we have an unrecognized type code
        pass
    asset_name = asset_d['name']
    asset_attributes = asset_d.get('attributes', {})
    
    if asset_id in asset_id_by_temporary_id:
        asset_id = asset_id_by_temporary_id[asset_id]
    asset = db.query(Asset).get(asset_id)
    if not asset:
        asset = Asset.make_unique_record(db)
        asset_id_by_temporary_id[asset_id] = asset.id
        asset_id = asset.id
    asset.type_code = asset_type_code
    asset.name = asset_name
    asset.attributes = asset_attributes
    
    print(asset.id, asset.type_code, asset.name)
    db.add(asset)
# db.commit()

2020-02-05 18:52:55,900 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset 
WHERE asset.id = ?
2020-02-05 18:52:55,902 INFO sqlalchemy.engine.base.Engine ('1',)
2020-02-05 18:52:55,905 INFO sqlalchemy.engine.base.Engine INSERT INTO asset (creation_datetime, modification_datetime, id, name, type_code, attributes) VALUES (?, ?, ?, ?, ?, ?)
2020-02-05 18:52:55,907 INFO sqlalchemy.engine.base.Engine ('2020-02-05 23:52:55.905335', None, '0z5jc2l1', None, None, None)
0z5jc2l1 AssetTypeCode.SUBSTATION Substation A
2020-02-05 18:52:55,911 INFO sqlalchemy.engine.base.Engine UPDATE asset SET name=?, type_code=?, attributes=? WHERE asset.id = ?
2020-02-05 18:52:55,912 INFO sqlalchemy.engine.base.Engine ('Substation A', 'SUBSTATION', <memory at 0x7fbef5a541f0>, '0z5

In [26]:
asset_id_by_temporary_id

{'1': '0z5jc2l1', '2': 'Rv3i7Sqk', '3': 'nQ2PpZvJ', '4': 'IK6eDAx8'}

In [27]:
asset.attributes

{'phaseCount': 3, 'lineType': 'mtx601', 'lineLength': 2000, 'lengthUnit': 'ft'}

In [28]:
# AssetTypeCode('x')

In [29]:
asset.type_code.value

'l'

In [30]:
db.query(Asset).count()

2020-02-05 18:53:23,242 INFO sqlalchemy.engine.base.Engine UPDATE asset SET name=?, type_code=?, attributes=? WHERE asset.id = ?
2020-02-05 18:53:23,244 INFO sqlalchemy.engine.base.Engine ('Line 1', 'LINE', <memory at 0x7fbef5c1dbb0>, 'IK6eDAx8')
2020-02-05 18:53:23,247 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset) AS anon_1
2020-02-05 18:53:23,248 INFO sqlalchemy.engine.base.Engine ()


4

In [31]:
# For each asset,
    # Save asset in database

In [32]:
from invisibleroads_macros_security import make_random_string
make_random_string(10)

'XohMdSyDkX'

In [33]:
from enum import Enum

class MyEnum(Enum):
    one = 1
    two = 2
    three = 3

MyEnum.one

<MyEnum.one: 1>

In [34]:
import enum
from sqlalchemy.types import Enum

class AssetTypeCode(enum.Enum):
    LINE = 'l'
    METER = 'm'
    TRANSFORMER = 't'
    SUBSTATION = 's'

In [35]:
asset_d = asset_dictionaries[1]
asset_d['connections']

[{'busId': 'Ai',
  'attributes': {'connectionType': 'delta',
   'baseVoltage': 115,
   'baseVoltageUnit': 'kV',
   'power': 5000,
   'powerUnit': 'kVA',
   'powerPercentResistance': 0.0005}},
 {'busId': 'Ao',
  'attributes': {'connectionType': 'wye',
   'baseVoltage': 4.16,
   'baseVoltageUnit': 'kV',
   'power': 5000,
   'powerUnit': 'kVA',
   'powerPercentResistance': 0.0005}}]

In [36]:
db.query(Connection).get({'asset_id': 1, 'bus_id': 2})

2020-02-05 18:53:28,564 INFO sqlalchemy.engine.base.Engine SELECT connection.asset_id AS connection_asset_id, connection.bus_id AS connection_bus_id, connection.attributes AS connection_attributes 
FROM connection 
WHERE connection.asset_id = ? AND connection.bus_id = ?
2020-02-05 18:53:28,566 INFO sqlalchemy.engine.base.Engine (1, 2)


In [37]:
from asset_tracker.models import Bus, Connection

bus_id_by_temporary_id = {}

for asset_d in asset_dictionaries:
    asset_id = asset_d['id']
    if asset_id in asset_id_by_temporary_id:
        asset_id = asset_id_by_temporary_id[asset_id]
    asset = db.query(Asset).get(asset_id)
    for connection_d in asset_d.get('connections', []):
        bus_id = connection_d['busId']
        # If we recognize a temporary id,
        if bus_id in bus_id_by_temporary_id:
            # Use the permanent id
            bus_id = bus_id_by_temporary_id[bus_id] 
        bus = db.query(Bus).get(bus_id)
        if not bus:
            bus = Bus.make_unique_record(db)
            # Link temporary id to permanent id
            bus_id_by_temporary_id[bus_id] = bus.id
            bus_id = bus.id
        db.add(bus)
        connection = db.query(Connection).get({'asset_id': asset_id, 'bus_id': bus_id})
        if not connection:
            connection = Connection(asset_id=asset_id, bus_id=bus_id)
        connection.attributes = connection_d.get('attributes', {})
        db.add(connection)
        # print(bus_id, connection.attributes)

2020-02-05 18:55:52,381 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset 
WHERE asset.id = ?
2020-02-05 18:55:52,382 INFO sqlalchemy.engine.base.Engine ('0z5jc2l1',)
2020-02-05 18:55:52,386 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset 
WHERE asset.id = ?
2020-02-05 18:55:52,387 INFO sqlalchemy.engine.base.Engine ('Rv3i7Sqk',)
2020-02-05 18:55:52,390 INFO sqlalchemy.engine.base.Engine SELECT bus.id AS bus_id 
FROM bus 
WHERE bus.id = ?
2020-02-05 18:55:52,391 INFO sqlalchemy.engine.base.Engine ('Ai',)
2020-02-05 18:

In [39]:
for asset in db.query(Asset):
    print('!!!', asset.id, asset.attributes, asset.connections)
    for connection in asset.connections:
        print(connection.attributes)

2020-02-05 18:56:40,347 INFO sqlalchemy.engine.base.Engine SELECT asset.creation_datetime AS asset_creation_datetime, asset.modification_datetime AS asset_modification_datetime, asset.id AS asset_id, asset.name AS asset_name, asset.type_code AS asset_type_code, asset.attributes AS asset_attributes 
FROM asset
2020-02-05 18:56:40,352 INFO sqlalchemy.engine.base.Engine ()
2020-02-05 18:56:40,354 INFO sqlalchemy.engine.base.Engine SELECT connection.asset_id AS connection_asset_id, connection.bus_id AS connection_bus_id, connection.attributes AS connection_attributes 
FROM connection 
WHERE ? = connection.asset_id
2020-02-05 18:56:40,355 INFO sqlalchemy.engine.base.Engine ('0z5jc2l1',)
!!! 0z5jc2l1 {} []
2020-02-05 18:56:40,358 INFO sqlalchemy.engine.base.Engine SELECT connection.asset_id AS connection_asset_id, connection.bus_id AS connection_bus_id, connection.attributes AS connection_attributes 
FROM connection 
WHERE ? = connection.asset_id
2020-02-05 18:56:40,359 INFO sqlalchemy.engin